In [1]:
import numba
from numba import cuda

In [2]:
import cupy
import numpy as np
import math
import time
import numba
from numba import cuda
from numba import njit
from numba import prange
import cudf
cupy.cuda.set_allocator(None)
#110.0, 100.0, 120.0, 0.35, 0.1, 0.05
N_PATHS = 8192000
Y_STEPS = 365 # constant, number of steps per year
T = 1.0 # time, unit 1 year
K = 110.0 # Strike price
B = 100.0 # barrier price
S0 = 120.0 # initial stock price 
sigma = 0.35 # stock annual volatility 
mu = 0.1 # stock annual return
r = 0.05 # stock annual interest rate
# calculate total step size
N_STEPS = int(np.ceil(T * Y_STEPS))
print('steps', N_STEPS)

steps 365


In [3]:
cupy.random.seed(11)

In [4]:
randoms_gpu = cupy.random.normal(0, 1, N_PATHS * N_STEPS, dtype=cupy.float32)
# randoms_cpu = np_randoms = cupy.asnumpy(randoms_gpu)
output =  np.zeros(N_PATHS, dtype=np.float32)

doutput =  np.zeros(N_PATHS*6, dtype=np.float32)

In [5]:
@cuda.jit
def numba_gpu_barrier_option(d_s, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    # ii - overall thread index
    
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp1 = mu/Y_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(1.0/Y_STEPS)
    running_average = 0.0
    for i in range(ii, N_PATHS, stride):
        s_curr = S0
        for n in range(N_STEPS):
            if n == N_STEPS - 1:
                delta_t = T - n/Y_STEPS
                tmp1 = delta_t * mu
                tmp3 = math.sqrt(delta_t)                
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average += (s_curr - running_average) / (n + 1.0)
            # print(running_average, n, tmp1 * s_curr, sigma,s_curr, tmp3,d_normals[i + n * N_PATHS])
            if running_average <= B:
                break
        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff

In [6]:
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cupy.zeros(N_PATHS, dtype=cupy.float32)
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
s = time.time()
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v)

time 0.4777834415435791 v 18.708305


In [7]:
(19.43269 - 19.432878)/2e-4





-0.9399999999892827

In [8]:
1/365

0.0027397260273972603

In [9]:
output

array([59.287468  ,  0.9394655 ,  4.4536147 , ...,  0.67556167,
       10.877099  ,  0.52545613], dtype=float32)

Calculate the differentiation by ADD method

The parameters are $T$, $K$, $S_0$, $\sigma$, $\mu$, $r$. The option price is computed by 
$$ p = E(f_i(\theta)) = \frac{1}{N}\sum_i f_i$$
where $f_i$ is the option value at the exercise time for the $i^{th}$ path.
$$\nabla_{\theta} p = \frac{1}{N}\sum_i \nabla_{\theta} f_i $$

Let's focus on the calculation of gradient of $f_i(\theta)$. $f_i$ is calculated by Monte Carlo simulation method. Break it down into steps. Without loss of generality, we drop the index $i$ here.

$$    \nabla_{\theta} f = 
\begin{cases}
    \nabla_{\theta} (a_n(\theta) - K)  & \text{if } a_n\geq K\\
    (0,0,0,0,0,0)              & \text{otherwise}
\end{cases}
$$

where the moving average $a_n$ at step $n$ is

$$a_n = g(a_{n-1}, s_{n}) = a_{n-1} + \frac{s_{n} - a_{n-1}}{n + 1.0}$$

The gradient of $a_n$:
$$ \nabla_{\theta} a_n = \frac{\partial g} {\partial a_{n-1}} \nabla_{\theta} a_{n-1} + \frac{\partial g} {\partial s_{n}} \nabla_{\theta} s_{n} = \frac{n}{n+1} \nabla_{\theta} a_{n-1} + \frac{1}{n+1} \nabla_{\theta} s_{n} $$

The stock price $s_n$ at step $n$ is:
$$s_n = s(s_{n-1}, \theta) = s_{n-1} + \frac{\mu}{Y} s_{n-1} + \sigma  \sqrt{\frac{1}{Y}} n_n s_{n-1}$$
At the last time step $s_n$ is:
$$s_n = s(s_{n-1}, \theta) = s_{n-1} + \frac{\mu (T Y - n) }{Y} s_{n-1} + \sigma  \sqrt{\frac{TY-n}{Y}} n_n s_{n-1}$$
The gradient of $s_n$:
$$\nabla_{\theta} s_n = \nabla_{\theta} s(s_{n-1}, \theta) = (1 + \frac{\mu}{Y} + \sigma \sqrt{\frac{1}{Y}} n_n) \nabla_{\theta} s_{n-1} + \nabla_{\theta} (1 + \frac{\mu}{Y} + \sigma \sqrt{\frac{1}{Y}} n_n) s_{n-1} $$

The initial contition $$\nabla_{\theta} S_0 = (0,0,1,0,0,0)$$

$$\nabla_{\theta} (1 + \frac{\mu}{Y} + \sigma \sqrt{\frac{1}{Y}} v_n) = (0, 0, 0, 1/Y, \sqrt{1/Y} n_n ,0) $$

$$\nabla_{\theta} (\frac{1+ \mu (T Y - n) }{Y} + \sigma  \sqrt{\frac{TY-n}{Y}} n_n ) = (\mu + \frac{1}{2}\sigma n_n (T-n/Y)^{-\frac{1}{2}}, 0, 0, (T-n/Y), \sqrt{T-n/Y} n_n ,0) $$

In [10]:
@cuda.jit
def numba_gpu_barrier_option(d_s, doutput, T, K, B, S0, sigma, mu, r, d_normals, N_STEPS, N_PATHS):
    # ii - overall thread index
    
    ii = cuda.threadIdx.x + cuda.blockIdx.x * cuda.blockDim.x
    stride = cuda.gridDim.x * cuda.blockDim.x
    tmp1 = mu/Y_STEPS
    tmp2 = math.exp(-r*T)
    tmp3 = math.sqrt(1.0/Y_STEPS)
    running_average = 0.0
    d_theta = numba.cuda.local.array(6, numba.float64)
    d_a = numba.cuda.local.array(6, numba.float64)
    for i in range(ii, N_PATHS, stride):
        d_theta[0] = 0 # T
        d_theta[1] = 0 # K
        d_theta[2] = 1 # S_0
        d_theta[3] = 0 # mu
        d_theta[4] = 0 # sigma
        d_theta[5] = 0 # r
        for k in range(6):
            d_a[k] = 0
        s_curr = S0
        for n in range(N_STEPS):
            if n == N_STEPS - 1:
                delta_t = T - n/Y_STEPS
                tmp1 = delta_t * mu
                tmp3 = math.sqrt(delta_t)  
            
            ## start to compute the gradient
            factor = (1.0+tmp1+sigma*tmp3*d_normals[i + n * N_PATHS])
            for k in range(6):
                 d_theta[k] *= factor
            if n == N_STEPS - 1:
                d_theta[0] += (mu + 0.5 * sigma * d_normals[i + n * N_PATHS] / tmp3) * s_curr
                d_theta[3] += (T - n/Y_STEPS) * s_curr
                d_theta[4] += tmp3 * d_normals[i + n * N_PATHS] * s_curr
            else:
                d_theta[3] += 1.0/Y_STEPS * s_curr
                d_theta[4] += tmp3 * d_normals[i + n * N_PATHS] * s_curr
            for k in range(6):
                d_a[k] = d_a[k]*n/(n+1.0) + d_theta[k]/(n+1.0)
            ## start to compute current stock price and moving average
              
            s_curr += tmp1 * s_curr + sigma*s_curr*tmp3*d_normals[i + n * N_PATHS]
            running_average += (s_curr - running_average) / (n + 1.0)
            # print(running_average, n, tmp1 * s_curr, sigma,s_curr, tmp3,d_normals[i + n * N_PATHS])
            if running_average <= B:
                break
        payoff = running_average - K if running_average>K else 0
        d_s[i] = tmp2 * payoff
        # gradient for strik 
        if running_average > K:
            d_a[1] = -1
            # adjust gradient for discount factor
            for k in range(6):
                d_a[k] *= tmp2
            d_a[0] += payoff * tmp2* -r 
            d_a[5] += payoff * tmp2* -T
        else:
            for k in range(6):
                d_a[k] = 0
        for k in range(6):
            doutput[k*N_PATHS+i] = d_a[k]

In [11]:
number_of_threads = 256
number_of_blocks = (N_PATHS-1) // number_of_threads + 1
output = cupy.zeros(N_PATHS, dtype=cupy.float32)
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, doutput, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
s = time.time()
numba_gpu_barrier_option[(number_of_blocks,), (number_of_threads,)](output, doutput, np.float32(T), np.float32(K), 
                    np.float32(B), np.float32(S0), 
                    np.float32(sigma), np.float32(mu), 
                    np.float32(r), randoms_gpu, N_STEPS, N_PATHS)
v = output.mean()
cuda.synchronize()
e = time.time()
print('time', e-s, 'v', v)
greeks = doutput.reshape(6, N_PATHS).mean(axis=1)
print('greeks', greeks)

time 1.8537318706512451 v 18.708305
greeks [ -0.9074474  -0.6714072   0.7713591  48.02249    20.488997  -18.708313 ]


In [12]:
import cupy
cupy_batched_barrier_option = cupy.RawKernel(r'''
extern "C" __global__ void batched_barrier_option(
    float *d_s,
    float *d_d,
    const float * T,
    const float * K,
    const float * B,
    const float * S0,
    const float * sigma,
    const float * mu,
    const float * r,
    const float * d_normals,
    const long *N_STEPS,
    const long Y_STEPS,
    const long N_PATHS,
    const long N_BATCH)
{
  unsigned idx =  threadIdx.x + blockIdx.x * blockDim.x;
  unsigned stride = blockDim.x * gridDim.x;
  unsigned tid = threadIdx.x;
  double d_theta[6];
  double d_a[6];

  for (unsigned i = idx; i<N_PATHS * N_BATCH; i+=stride)
  {
    d_theta[0] = 0; // T
    d_theta[1] = 0; // K
    d_theta[2] = 1.0; // S_0
    d_theta[3] = 0; // mu
    d_theta[4] = 0; // sigma
    d_theta[5] = 0; // r
    for (unsigned k = 0; k < 6; k++){
      d_a[k] = 0.0;
    }
    
    int batch_id = i / N_PATHS;
    int path_id = i % N_PATHS;
    float s_curr = S0[batch_id];
    float tmp1 = mu[batch_id]/Y_STEPS;
    float tmp2 = exp(-r[batch_id]*T[batch_id]);
    float tmp3 = sqrt(1.0/Y_STEPS);
    unsigned n=0;
    double running_average = 0.0;
    for(unsigned n = 0; n < N_STEPS[batch_id]; n++){
        if (n == N_STEPS[batch_id] - 1) {
            float delta_t = T[batch_id] - n/Y_STEPS;
            tmp1 = delta_t * mu[batch_id];
            tmp3 = sqrt(delta_t);
        }
        float normal = d_normals[path_id + batch_id * N_PATHS + n * N_PATHS * N_BATCH];
        
            
        // start to compute the gradient
        float factor = (1.0+tmp1+sigma[batch_id]*tmp3*normal);
        for (unsigned k=0; k < 6; k++) {
            d_theta[k] *= factor;
        }
        
        if (n == N_STEPS[batch_id] - 1){
                d_theta[0] += (mu[batch_id] + 0.5 * sigma[batch_id] * normal / tmp3) * s_curr;
                d_theta[3] += (T[batch_id] - n/Y_STEPS) * s_curr;
                d_theta[4] += tmp3 * normal * s_curr;
        }
        else {
                d_theta[3] += 1.0/Y_STEPS * s_curr;
                d_theta[4] += tmp3 * normal * s_curr;
        }
        for (unsigned k = 0; k < 6; k++) {
                d_a[k] = d_a[k]*n/(n+1.0) + d_theta[k]/(n+1.0); 
        }
        
        
        // start to compute current stock price and moving average       
       
       s_curr += tmp1 * s_curr + sigma[batch_id]*s_curr*tmp3*normal;
       running_average += (s_curr - running_average) / (n + 1.0);
       if (running_average <= B[batch_id]){
           break;
       }
    }

    float payoff = (running_average>K[batch_id] ? running_average-K[batch_id] : 0.f); 
    d_s[i] = tmp2 * payoff;
    
    // gradient for strik 
    if (running_average > K[batch_id]){
       d_a[1] = -1.0;
       // adjust gradient for discount factor
       for (unsigned k = 0; k < 6; k++) {
            d_a[k] *= tmp2;
        }
        d_a[0] += payoff * tmp2* -r[batch_id];
        d_a[5] += payoff * tmp2* -T[batch_id];
        
    }
    else {
        for (unsigned k = 0; k < 6; k++) {
           d_a[k] = 0.0;
        }

    }
    
    for (unsigned k = 0; k < 6; k++) {
       d_d[k*N_PATHS*N_BATCH+i] = d_a[k];
    }
  }
}

''', 'batched_barrier_option')

In [13]:

import numpy
cupy.ceil((cupy.random.rand(10)*5)).astype(cupy.int64)

array([1, 4, 5, 1, 1, 2, 3, 2, 1, 1])

In [14]:
N_PATHS = 8192000
Y_STEPS = 262
N_BATCH = 2
T = 1.0
#110.0, 100.0, 120.0, 0.35, 0.1, 0.05
K = cupy.array([110.0, 120.0], dtype=cupy.float32)
B = cupy.array([100.0, 90.0], dtype=cupy.float32)
S0 = cupy.array([120.0, 100.0], dtype=cupy.float32)
sigma = cupy.array([0.35, 0.2], dtype=cupy.float32)
mu = cupy.array([0.1, 0.1], dtype=cupy.float32)
r =cupy.array([0.05, 0.05], dtype=cupy.float32)
T =cupy.array([1.0, 1.1], dtype=cupy.float32)
N_STEPS = cupy.ceil(T * Y_STEPS).astype(cupy.int64)

In [15]:
import time
def batch_run():
    number_of_threads = 256
    number_of_blocks = (N_PATHS * N_BATCH - 1) // number_of_threads + 1
    random_elements = (N_STEPS.max()*N_PATHS*N_BATCH).item()
    randoms_gpu = cupy.random.normal(0, 1, random_elements, dtype=cupy.float32)
    output = cupy.zeros(N_BATCH*N_PATHS, dtype=cupy.float32)
    d_output = cupy.zeros(N_BATCH*N_PATHS*6, dtype=cupy.float32)
    cupy.cuda.stream.get_current_stream().synchronize()
    s = time.time() 
    cupy_batched_barrier_option((number_of_blocks,), (number_of_threads,),
                       (output, d_output, T, K, B, S0, sigma, mu, r,
                        randoms_gpu, N_STEPS, Y_STEPS, N_PATHS, N_BATCH))
    v = output.reshape(N_BATCH, N_PATHS).mean(axis=1)
    b = d_output.reshape(6, N_BATCH, N_PATHS).mean(axis=2)
    cupy.cuda.stream.get_current_stream().synchronize()
    e = time.time()
    print('time', e-s, 'v',v)
    print(b.shape)
    print('gradient', b)
    return output
o = batch_run()

time 2.6104929447174072 v [18.756649   1.0737803]
(6, 2)
gradient [[-8.9926428e-01 -4.6959769e-02]
 [-6.7189366e-01 -1.3079715e-01]
 [ 7.7224147e-01  1.6769104e-01]
 [ 4.8513054e+01  9.8589897e+00]
 [ 2.0489370e+01  1.5784095e+01]
 [-1.8756649e+01 -1.1811584e+00]]


In [16]:
N_PATHS

8192000

In [17]:
(18.760712 - 18.71775)/0.002

21.481000000001416

In [18]:
[ -0.9074474,  -0.6714072   0.7713591  48.02249    20.488997  -18.708313 ]

SyntaxError: invalid syntax (<ipython-input-18-9f78a3372395>, line 1)